### KOTE test

In [3]:
import pandas as pd
from datasets import load_dataset
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline


In [5]:
tokenizer = AutoTokenizer.from_pretrained("searle-j/kote_for_easygoing_people")
model = AutoModelForSequenceClassification.from_pretrained("searle-j/kote_for_easygoing_people")

# 감정 분석 모델 설정 
pipe = TextClassificationPipeline(
        model=model,
        tokenizer=tokenizer,
        device=0, # gpu number, -1 if cpu used
        return_all_scores=True,
        function_to_apply='sigmoid'
    )

Device set to use cuda:0


In [24]:
df = pd.read_csv("new_merge.csv")
df

,title,artist,like_cnt,lyrics
0,"HOME SWEET HOME (feat. 태양, 대성)",G-DRAGON,"2,220",You say Its changed Show must go on Behave 오랜만...
1,POWER,G-DRAGON,"1,792",When GDs in the house bermensch When GDs in th...
2,Igloo,KISS OF LIFE,790,Imma back up every word Mini skirt pretty pink...
3,Small girl (feat. 도경수(D.O.)),이영지,"3,308",If I got a two small cheeks and a bright pink ...
4,SPOT! (feat. JENNIE),지코 (ZICO),"2,574",Everything ok my man Turn the music up Uhm we...
...,...,...,...,...
1391,연,오반(OVAN),22,우 어쩌면 우리는 Babe 우리도 모르게 Babe 서로를 계속 원했을까 아마...
1392,살아간다,최유리,616,나는 조금만 더 올라가면 보일걸 그 말을 굳게 믿은 채 다시 살아간다 나는 조그...
1393,Smoking Dreams,재지팩트(Jazzyfact),"3,969",i puff onetime and i puff twice on and on an...
1394,사랑했던 우리는 없어,리,12,뜨겁게 사랑했던 우리는 없어 한없이 행복했던 우리는 없어 노력해도 안되는 답이...


In [23]:
# 데이터 로드
text = df['lyrics'].tolist()

# 긴 텍스트 자르기 함수
def truncate_texts(texts, tokenizer, max_length):   # 입력 텍스트 리스트를 받아, 각 텍스트를 최대 길이로 자르고 잘라낸 텍스트를 반환합니다.
    truncated_texts = []
    for t in texts:
        tokens = tokenizer(t, max_length=max_length, truncation=True)       #텍스트 t를 토큰화할 때, 최대 길이 max_length로 잘라내고 필요하면 truncation=True로 설정하여 초과 부분을 제거합니다.
        truncated_texts.append(tokenizer.decode(tokens['input_ids'], skip_special_tokens=True))     #잘라낸 토큰을 다시 텍스트로 변환하며, 특수 토큰은 제거합니다.
    return truncated_texts

# 텍스트 리스트를 잘라서 파이프라인에 전달
max_length = tokenizer.model_max_length  # 일반적으로 512
truncated_texts = truncate_texts(text, tokenizer, max_length)   # text 리스트의 각 텍스트를 최대 길이로 잘라냅니다.
pip_text = pipe(truncated_texts)        #잘라낸 텍스트를 파이프라인에 전달하여 감정 분석을 수행합니다.


# 결과 저장을 위한 리스트
results = []

# 각 감정에 해당할 확률(score)이 0.4 이상인 감정들만 저장
for i, t in enumerate(truncated_texts):            # 잘라낸 텍스트와 그에 대한 감정 분석 결과를 반복합니다.
    for output in pip_text[i]:
        if output["score"] > 0.4:
            results.append({
                "text": t,
                "label": output["label"],
                "score": output["score"]
            })

# 결과를 데이터프레임으로 변환
results_df = pd.DataFrame(results)

# 데이터프레임 저장 
results_df.to_csv("kote_result.csv", index=False)

# # 각 감정에 해당할 확률(score)이 0.4 이상인 감정들만 출력
# for i, t in enumerate(truncated_texts):     # 잘라낸 텍스트와 그에 대한 감정 분석 결과를 반복합니다.
#     print(f"Text: {t}")
#     for output in pip_text[i]:
#         if output["score"] > 0.4:
#             print(output)
#     print("\n")